In [58]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt
import traceback
import time
from datetime import datetime
import cv2

In [59]:
tlf = py.TlFactory.GetInstance()
cam = py.InstantCamera(tlf.CreateFirstDevice())
cam.Open()

In [60]:
cam.UserSetSelector = 'Default'
cam.UserSetLoad.Execute()

In [61]:
width = 1080
height = 1920
fps = 30

In [63]:
cam.ExposureAuto = 'Off'
cam.PixelFormat = 'YCbCr422_8'
cam.ExposureTime = 30000
cam.AcquisitionFrameRate = fps

In [64]:
converter = py.ImageFormatConverter()
converter.OutputPixelFormat = py.PixelType_RGB8packed
converter.OutputBitAlignment = "MsbAligned"

In [65]:
def draw_time_date(frame):

    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (1150, 50)
    fontScale = 1
    color = (255, 255, 0)
    thickness = 2

    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    frame = cv2.putText(frame, dt_string, org, font, fontScale, color, thickness, cv2.LINE_AA)

    # Attempt to display using cv2 (doesn't work)
    cv2.namedWindow("Input")
    cv2.imshow("Input", frame)
    cv2.waitKey(1)
    return frame

In [74]:
output_video = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), fps, (width,height))

def save_video(frame):
    frame = np.frombuffer(frame, dtype=np.uint8).reshape(height, width, -1)
    frame = cv2.cvtColor(cv2.resize(frame, (height,width)), cv2.COLOR_RGB2BGR)

    frame = draw_time_date(frame)
    # output_video.write(frame)


    return

In [72]:
class ImageHandler (py.ImageEventHandler):
    def __init__(self, *args):
        super().__init__(*args)
        self.time_old = time.time()
    
    def OnImageGrabbed(self, camera, grabResult):
        try:
            if grabResult.GrabSucceeded():
                
                if (~converter.ImageHasDestinationFormat(grabResult)):
                    grabResult = converter.Convert(grabResult)
                    
                img = grabResult.Array
                time_new = time.time()
                rate = 1/(time_new-self.time_old)
                print(rate)
                self.time_old = time_new
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB )
                # cv2.namedWindow("Input")
                # cv2.imshow("Input", img)
                # cv2.waitKey(1)
                save_video(img)
            else:
                raise RuntimeError("Grab failed")
        except Exception as e:
            traceback.print_exc()

In [68]:
def BackgroundLoopSample():
    handler = ImageHandler()

    cam.RegisterImageEventHandler(handler, py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # cam.StartGrabbingMax(100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    cam.StartGrabbing(py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)

    try:
        while cam.IsGrabbing():
            pass
    except KeyboardInterrupt:
        pass

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)
    cv2.destroyAllWindows()

    # return handler.img_sum

In [75]:
BackgroundLoopSample()

7.596037452233913
18.568069131605043
32.34349167180753
29.491246080072
26.382754955056956
25.710632298403162
29.492075545992773
30.384920203710546
30.384920203710546
32.34199528090927
35.808047262513554
25.067708987024783
37.13482310443744
33.424211272880854
31.324152352501866
32.35147476243367
43.60345974717232
33.42048270531709
32.34399049954503
28.648052019015353
23.312438513314472
38.57966482091282
34.575088615942626
31.33327855013783
28.648247693074783
28.167272190024647
27.852473603824954
33.422346885110045
30.383159358769113
31.303813056490554
30.412900980335287
26.385908404630097
21.801049950621135
30.388882851160332
30.38403975572829
28.647660678915376
31.33117203256891
31.33538535098467
33.424211272880854
32.336011101688385
34.58278571605254
35.81049306296691
27.097787885052718
32.34723325492615
28.03210693400167
29.490001968670022
32.34972812463846
28.66978817063918
32.08371452612254
29.491868175138343
29.51801988838296
26.36086757043825
33.122775983384535
28.564159141366677

In [39]:
cam.IsOpen()

True

In [57]:
cam.Close()